In [1]:
import pandas as pd
import fiona
import geopandas as gpd
from openpyxl import load_workbook

## Load WalkMA Excel Data

In [3]:
import pandas as pd
from openpyxl import load_workbook

# Load dataframe
df = pd.read_excel("data/Walk Audit Database.xlsx", header=1)

# Load workbook
wb = load_workbook("data/Walk Audit Database.xlsx", data_only=True)
ws = wb.active  # or wb["SheetName"]

# Locate VIEW column (Excel is 1-based)
view_col_idx = df.columns.get_loc("VIEW") + 1

# Excel row where data starts
excel_start_row = 3  # header=1 → data starts at row 3

links = []
for i in range(len(df)):
    excel_row = excel_start_row + i
    cell = ws.cell(row=excel_row, column=view_col_idx)
    links.append(cell.hyperlink.target if cell.hyperlink else None)

# Add links
df["VIEW_link"] = links

## Load Geodata

In [4]:
gdb_path = "data/townssurvey_gdb/townssurvey_gdb/townssurvey.gdb"

In [5]:
layers = fiona.listlayers(gdb_path)
layers

['TOWNSSURVEY_ARC_GENCOAST',
 'TOWNSSURVEY_POLYM_GENCOAST',
 'TOWNSSURVEY_POLY',
 'TOWNSSURVEY_PT',
 'TOWNSSURVEY_POLYM',
 'TOWNSSURVEY_ARC',
 'T_1_DirtyAreas',
 'T_1_PointErrors',
 'T_1_LineErrors',
 'T_1_PolyErrors',
 'TOWNS_POLY_INCGW',
 'TOWNS_POLY_AREACODE']

In [6]:
layer_name = "TOWNSSURVEY_POLYM"
gdf = gpd.read_file(gdb_path, layer=layer_name, engine="pyogrio")

In [7]:
len(gdf)

351

## Cleaning

In [8]:
# Clean df
df['CITY/TOWN'] = df['CITY/TOWN'].str.strip()  # Remove whitespace
df['CITY/TOWN'] = df['CITY/TOWN'].str.upper()  # Standardize case

In [9]:
# Remove parentheses and content from CITY/TOWN column
df['CITY'] = df['CITY/TOWN'].str.replace(r'\s*\(.*?\)', '', regex=True)

# Extract neighborhood from parentheses into new column
df['NEIGHBORHOOD'] = df['CITY/TOWN'].str.extract(r'\((.*?)\)', expand=False)

In [10]:
# Clean gdf
gdf['TOWN'] = gdf['TOWN'].str.strip()
gdf['TOWN'] = gdf['TOWN'].str.upper()
gdf = gdf.rename(columns={'TOWN': 'CITY'})

# set CRS
gdf = gdf.to_crs(epsg=4326)

In [11]:
merged_df = pd.merge(df, gdf, on='CITY', how='left')

# drop cols
merged_df = merged_df.drop(columns=['FIPS_STCO', 'FOURCOLOR',
       'POP1960', 'POP1970', 'POP1980', 'POP1990', 'POP2000', 'POP2010',
       'POP2020', 'POPCH10_20', 'AREA_ACRES', 'AREA_SQMI', 'SHAPE_Length',
       'SHAPE_Area'])

# make geo
merged_gdf = gpd.GeoDataFrame(merged_df)

In [12]:
len(merged_gdf)

111

In [13]:
len(df)

111

In [14]:
# not captured
# merged_df[merged_df['geometry'].isna()]

In [15]:
gdf.to_file(
    "data/out/towns.geojson",
    driver="GeoJSON",
)

merged_gdf.to_file(
    "data/out/audits.geojson",
    driver="GeoJSON",
)

# copy data
! cp data/out/towns.geojson app/public/data
! cp data/out/audits.geojson app/public/data

### Prepare sample

In [18]:
sample_towns = gdf.sample(2)
sample_audits = merged_gdf.sample(2)

sample_towns.to_file(
    "data/out/sample_towns.geojson",
    driver="GeoJSON",
)

sample_audits.to_file(
    "data/out/sample_audits.geojson",
    driver="GeoJSON",
)